# 手で作ったサンプルデータを用いて、交絡因子と交互作用項をロジスティック回帰モデルで検討する

In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 100)

# 交絡因子

In [3]:
#サンプルデータの読み込み
df_conf = pd.read_csv('./data/sample_data_confounding.csv')

#切片用のカラムを追加
df_conf["intercept"] = 1
df_conf.head()

,id,alcohol,smoking,cancer,intercept
0,1,1,1,1,1
1,2,1,1,1,1
2,3,1,1,1,1
3,4,1,1,1,1
4,5,1,1,1,1


In [4]:
pd.crosstab(df_conf['alcohol'], df_conf['cancer'])

cancer,0,1
alcohol,,
0,244,52
1,239,102


## 単変数ロジスティック回帰（粗オッズ）

In [5]:
X = df_conf[ ['intercept', 'alcohol'] ]
y = df_conf['cancer']

In [6]:
logit = sm.GLM(y, X, family=sm.families.Binomial())
logit_result = logit.fit()
print( logit_result.summary() )

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 cancer   No. Observations:                  637
Model:                            GLM   Df Residuals:                      635
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -345.62
Date:                Fri, 09 Jul 2021   Deviance:                       691.25
Time:                        06:28:51   Pearson chi2:                     637.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.5459      0.153    -10.121      0.0

# ２変数ロジスティック回帰モデル（喫煙有無による調整）

In [7]:
X = df_conf[ ['intercept', 'alcohol', 'smoking'] ]
y = df_conf['cancer']

In [8]:
logit = sm.GLM(y, X, family=sm.families.Binomial())
logit_result = logit.fit()
print( logit_result.summary() )

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 cancer   No. Observations:                  637
Model:                            GLM   Df Residuals:                      634
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -332.22
Date:                Fri, 09 Jul 2021   Deviance:                       664.44
Time:                        06:49:32   Pearson chi2:                     637.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.8664      0.174    -10.738      0.0

In [ ]:
df1['alcohol*smoking'] = df1['alcohol'] * df1['smoking']

In [ ]:
X = df1[ ['intercept', 'alcohol', 'smoking', 'alcohol*smoking'] ]
y = df1['cancer']

In [ ]:
logit = sm.GLM(y, X, family=sm.families.Binomial())
logit_result = logit.fit()
print( logit_result.summary() )